In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
import QUANTAXIS as QA
import talib as ta
import datetime #, time

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

挑两只股票进行对比

比如AMZN和MSFT

获取两只股票的价格

正序，不要搞逆序了，233

想到了再说

算ratio

循环算side

In [3]:
code = ['AMZN', 'MSFT']
start = '2010-01-01'
end = '2018-12-19'

In [9]:
AMZN_data = QA.QA_fetch_get_usstock_day('tdx', code[0], start, end)
MSFT_data = QA.QA_fetch_get_usstock_day('tdx', code[1], start, end)

In [13]:
MSFT_data.head()

,open,high,low,close,position,trade,price,amount,date,code,date_stamp
date,,,,,,,,,,,
2010-01-04,30.620001,31.100000,30.590000,30.950001,0,384091,0.0,0.0,2010-01-04,MSFT,1.262534e+09
2010-01-05,30.850000,31.100000,30.639999,30.959999,0,497496,0.0,0.0,2010-01-05,MSFT,1.262621e+09
2010-01-06,30.879999,31.080000,30.520000,30.770000,0,581824,0.0,0.0,2010-01-06,MSFT,1.262707e+09
2010-01-07,30.629999,30.700001,30.190001,30.450001,0,505597,0.0,0.0,2010-01-07,MSFT,1.262794e+09
2010-01-08,30.280001,30.879999,30.240000,30.660000,0,511974,0.0,0.0,2010-01-08,MSFT,1.262880e+09


In [12]:
AMZN_close = AMZN_data.close
MSFT_close = MSFT_data.close

In [15]:
ratio = AMZN_close / MSFT_close

In [38]:
position_side_list = []
for i in range(ratio.shape[0]):
    if i > 1:
        if ratio[i] > np.mean(ratio[:i-1]):
            position_side_list.append(1)
        elif ratio[i] < np.mean(ratio[:i-1]):
            position_side_list.append(-1)
        else:
            position_side_list.append(np.nan)
    elif i == 1:
        if ratio[1] > ratio[0]:
            position_side_list.append(1)
        elif ratio[1] < ratio[0]:
            position_side_list.append(-1)
        else:
            position_side_list.append(np.nan)
    elif i == 0:
        position_side_list.append(np.nan)
position_side_series = pd.Series(position_side_list, ratio.index)

In [37]:
ratio[2255]

True

In [26]:
position_side_series.head()

date
2010-01-04    NaN
2010-01-05    1.0
2010-01-06   -1.0
2010-01-07   -1.0
2010-01-08    1.0
dtype: float64

In [39]:
position_side_series.sum()

1799.0

In [40]:
ratio.sum()

22457.885555324698

In [28]:
position_side_series.shift(1).head()

date
2010-01-04    NaN
2010-01-05    NaN
2010-01-06    1.0
2010-01-07   -1.0
2010-01-08   -1.0
dtype: float64

In [29]:
backtest_position_side = position_side_series.shift(1)

In [19]:
ratio.head()

date
2010-01-04    4.326332
2010-01-05    4.350452
2010-01-06    4.298017
2010-01-07    4.269294
2010-01-08    4.354860
Name: close, dtype: float64

In [27]:
AMZN_close.pct_change().head()

date
2010-01-04         NaN
2010-01-05    0.005900
2010-01-06   -0.018116
2010-01-07   -0.017013
2010-01-08    0.027077
Name: close, dtype: float64

In [30]:
AMZN_pct = AMZN_close.pct_change()
MSFT_pct = MSFT_close.pct_change()

In [33]:
((1+backtest_position_side)/2 * AMZN_pct + \
 (1-backtest_position_side)/2 * MSFT_pct).mean()

0.0009430844400986199

In [64]:
(backtest_position_side * (AMZN_pct - MSFT_pct)).mean()

-1.977600631827883e-05

In [46]:
ratio > ratio.rolling(14).mean()

date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
2010-01-11         NaN
2010-01-12         NaN
2010-01-13         NaN
2010-01-14         NaN
2010-01-15         NaN
2010-01-19         NaN
2010-01-20         NaN
2010-01-21         NaN
2010-01-22    4.232400
2010-01-25    4.216472
2010-01-26    4.195024
2010-01-27    4.183535
2010-01-28    4.187301
2010-01-29    4.194119
2010-02-01    4.185488
Name: close, dtype: float64

In [68]:
backtest_position_side_rolling = pd.Series(np.where(ratio>ratio.\
            rolling(14).mean(), 1, -1).tolist(), ratio.index).shift(1)

In [54]:
backtest_position_side_rolling.head()

date
2010-01-04   -1
2010-01-05   -1
2010-01-06   -1
2010-01-07   -1
2010-01-08   -1
dtype: int64

In [62]:
((1+backtest_position_side_rolling)/2 * AMZN_pct + \
 (1-backtest_position_side_rolling)/2 * MSFT_pct).mean()

0.0007901934840344638

In [69]:
(backtest_position_side_rolling * (AMZN_pct - MSFT_pct)).mean()

-0.00032747237168519866

我们来试下港股的效果

In [19]:
code = ['00939', '01398'] # 建设银行 & 工商银行
start = '2016-01-10'
end = '2018-12-04'

In [20]:
JS_data = QA.QA_fetch_get_hkstock_day('tdx', code[0], start, end)
GS_data = QA.QA_fetch_get_hkstock_day('tdx', code[1], start, end)

In [26]:
JS_data.head()

,open,high,low,close,position,trade,price,amount,date,code,date_stamp
date,,,,,,,,,,,
2016-01-11,4.94,4.96,4.90,4.92,1320919419,31917,27151000.0,1.573961e+09,2016-01-11,00939,1.452442e+09
2016-01-12,4.97,4.98,4.88,4.89,1318456089,25601,30595000.0,1.258655e+09,2016-01-12,00939,1.452528e+09
2016-01-13,4.94,5.01,4.88,4.89,1318730031,26155,24641000.0,1.293719e+09,2016-01-13,00939,1.452614e+09
2016-01-14,4.82,4.87,4.79,4.82,1321579691,34342,28318000.0,1.658476e+09,2016-01-14,00939,1.452701e+09
2016-01-15,4.80,4.84,4.72,4.73,1322327211,36841,22516000.0,1.754158e+09,2016-01-15,00939,1.452787e+09


In [27]:
GS_data.head()

,open,high,low,close,position,trade,price,amount,date,code,date_stamp
date,,,,,,,,,,,
2016-01-11,4.22,4.25,4.20,4.22,1317920017,28169,18863000.0,1.190038e+09,2016-01-11,01398,1.452442e+09
2016-01-12,4.25,4.28,4.13,4.16,1319675885,33728,22086000.0,1.414789e+09,2016-01-12,01398,1.452528e+09
2016-01-13,4.22,4.28,4.15,4.17,1315884116,23793,16959000.0,1.001592e+09,2016-01-13,01398,1.452614e+09
2016-01-14,4.10,4.20,4.08,4.15,1314281672,21754,15575000.0,8.990356e+08,2016-01-14,01398,1.452701e+09
2016-01-15,4.15,4.16,4.05,4.05,1315759489,24318,13315000.0,9.936159e+08,2016-01-15,01398,1.452787e+09


through pandas_reader

In [10]:
import pandas_datareader as web
import datetime

In [23]:
start_date = datetime.datetime.strptime(start, '%Y-%m-%d')
end_date = datetime.datetime.strptime(end, '%Y-%m-%d')
js_data_from_yahoo = web.DataReader(code[0][1:]+'.hk', 'yahoo', start_date, end_date)
gs_data_from_yahoo = web.DataReader(code[1][1:]+'.hk', 'yahoo', start_date, end_date)

In [24]:
js_data_from_yahoo.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-11,4.96,4.90,4.94,4.92,319178128.0,4.215010
2016-01-12,4.98,4.88,4.97,4.89,256018843.0,4.189308
2016-01-13,5.01,4.88,4.94,4.89,261559608.0,4.189308
2016-01-14,4.87,4.79,4.82,4.82,343420062.0,4.129339
2016-01-15,4.84,4.72,4.80,4.73,368419941.0,4.052235


In [25]:
gs_data_from_yahoo.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-11,4.25,4.20,4.22,4.22,281691602.0,3.566770
2016-01-12,4.28,4.13,4.25,4.16,337288458.0,3.516058
2016-01-13,4.28,4.15,4.22,4.17,237935728.0,3.524510
2016-01-14,4.20,4.08,4.10,4.15,217544037.0,3.507606
2016-01-15,4.16,4.05,4.15,4.05,243188163.0,3.423086
